# Similaridade de Strings 
- Data cleaning
- Correção de digitação 
- Tradução de idiomas






In [2]:
!pip install fuzzywuzzy

In [3]:
!pip install fuzzywuzzy[speedup]

   ---------------------------------------- 0.0/98.4 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/98.4 kB ? eta -:--:--
   ------------------------ --------------- 61.4/98.4 kB 1.1 MB/s eta 0:00:01
   ------------------------------------- -- 92.2/98.4 kB 1.1 MB/s eta 0:00:01
   ------------------------------------- -- 92.2/98.4 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 98.4/98.4 kB 564.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.7 MB 7.0 MB/s eta 0:00:01
   -------- ------------------------------- 0.3/1.7 MB 3.6 MB/s eta 0:00:01
   ------------ --------------------------- 0.5/1.7 MB 3.6 MB/s eta 0:00:01
   ------------------ --------------------- 0.8/1.7 MB 4.3 MB/s eta 0:00:01
   ------------------------- -------------- 1.1/1.7 MB 4.8 MB/s eta 0:00:01
   -------------------------------- ------- 1.4/1.7 MB 5.0 MB/s eta 0:00:01
   -----------------

In [4]:
from fuzzywuzzy import fuzz,process

Aplicando fuzzywuzzy em strings

In [9]:
s1 = "Doença Cardiovascular"
s2= "Doença Cardiovascular"
fuzz.ratio(s1,s2)

100

In [10]:
s1 = "Doença Cardiovascular"
s2= "Doença Cardiovasculhar"
fuzz.ratio(s1,s2)

98

Comportamento sobre letras maiusculas

In [12]:
s1 = "Doença Cardiovascular"
s2= "doença Cardiovascular"
fuzz.ratio(s1,s2)

95

Comportamento sobre pontuação

In [15]:
s1 = "Doença Cardiovascular"
s2= "Doença Cardiovascular!!"
fuzz.ratio(s1,s2)

95

# Similaridade parcial
- Similaridade parcial busca apenas a string em questão e descarta o resto
- Extremamente util para trabalha com dados coletados da web

In [19]:
#Consulta o score usando o metodo ratio
s1 = 'Doençã Cardiovascular'
s2 = '!#&*#&!&Doença Cardiovascular¨*%&$%¨$'
fuzz.ratio(s1,s2)

67

In [26]:
#Consulta o score usando o metodo partial
s1 = 'Doençã Cardiovascular'
s2 = '!#&*#&!&Doençâ Cardiovascular¨*%&$%¨$'
fuzz.partial_ratio(s1,s2)

95

In [25]:
#Consulta o score usando o metodo partial
#Alteração na STRING
s1 = 'Doençã Cardiovascular'
s2 = '!#&*#&!&Doença CardiovasculHar¨*%&$%¨$'
fuzz.partial_ratio(s1,s2)

90

In [23]:
s1 = "Doenças Cardiovascular"
s2 = "Cardiovascular Doenças"
fuzz.partial_ratio(s1,s2)

64

- Funçãos partial_token_sort_ratio() separa os tokens por espaço e ordena por ordem alfabetica 
- Coloca as Strings em letras mminusculas
- Considera apenas as Strings consultadas

In [28]:
s1 = "Doenças Cardiovascular"
s2 = "Cardiovascular Doenças"
fuzz.partial_token_sort_ratio(s1,s2)

100

# Processando uma Lista de Strings
- Aplicar o fuzzywuzzy para corrigir strings em uma base de dados

In [29]:
# Criando lista de strings
lista = ['Doença Cardiovascular','doença cardiovascular!!','Doenca Cardiovascular','Doenc. Cardio']

Extrair os score de similaridades com uma string em questão

In [30]:
process.extract('Doença Cardiovascular',lista,scorer=fuzz.partial_ratio)

[('Doença Cardiovascular', 100),
 ('doença cardiovascular!!', 100),
 ('Doenca Cardiovascular', 95),
 ('Doenc. Cardio', 85)]

Extraindo apenas Strings cm score acima de 95

In [31]:
process.extractOne('Doença Cardiovascular',lista,scorer=fuzz.partial_ratio,score_cutoff=95)#extractOne extrai apenas 1 string

('Doença Cardiovascular', 100)

# Data Cleaning em um DataFrame
- Aplicando o fuzzywuzzy em uma base de dados
- Medir a similaridade de strings e fazer __Data Cleaning__

In [59]:
import pandas as pd 
from collections import OrderedDict
data = OrderedDict(
    {
        'codigo_produto': [10,11,12,13,14],
        'descrição':['iphone 6ss','iphone 6s','ipone 6s', 'iphoni 6s','Iphone 6s,,,,,']
    }
)

In [60]:
dataset = pd.DataFrame(data)

In [61]:
dataset

,codigo_produto,descrição
0,10,iphone 6ss
1,11,iphone 6s
2,12,ipone 6s
3,13,iphoni 6s
4,14,"Iphone 6s,,,,,"


In [49]:
process.extractOne('Iphone 6s',choices=dataset.descrição,scorer=fuzz.ratio,score_cutoff=96)

('iphone 6s', 100, 1)

Função que aplica o metodo fuzzy

In [62]:
def aplicafuzzy (query, dados,metodo_ratio,score_corte):
    return process.extractOne(query,choices=dados,scorer=metodo_ratio,score_cutoff=score_corte)

In [52]:
aplicafuzzy('Iphone 6s',dataset.descrição,fuzz.ratio,95)

('iphone 6s', 100, 1)

Criando uma nova coluna no Dataset a partir das strings similares

In [65]:
dataset['descrição2'] = aplicafuzzy('iphone 6s',dataset.descrição,fuzz.ratio,95)[0]

In [64]:
dataset

,codigo_produto,descrição,descrição2
0,10,iphone 6ss,iphone 6s
1,11,iphone 6s,iphone 6s
2,12,ipone 6s,iphone 6s
3,13,iphoni 6s,iphone 6s
4,14,"Iphone 6s,,,,,",iphone 6s


In [56]:
process.extract('Iphone 6s',dataset['descrição'],scorer=fuzz.partial_ratio)

[('iphone 6ss', 100, 0),
 ('iphone 6s', 100, 1),
 ('Iphone 6s,,,,,', 100, 4),
 ('iphoni 6s', 89, 3),
 ('ipone 6s', 88, 2)]